## Imports

In [ ]:
# pip install --upgrade scikit-learn

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.3
Sklearn  0.24.1


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = True

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "joshbatt"
    os.environ['KAGGLE_KEY']      = "a5effb5656a3fb38b287707b1576bc1e"  # See https://www.kaggle.com/docs/api
    #!pip install --upgrade kaggle
    #!kaggle competitions download -c titanic
    DATA_PATH = "titanic/"

else:
    DATA_PATH = "titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [4]:
print("PRINT DATASET")
df.isnull().sum()

PRINT DATASET


Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [5]:
print("PRINT TEST DATASET (NO SURVIVOR COLUMN)")
df_test.isnull().sum()

PRINT TEST DATASET (NO SURVIVOR COLUMN)


Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [6]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda x: x.split(',')[1].split('.')[0].strip()

# YOUR CODE HERE
#raise NotImplementedError()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [7]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [8]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [9]:
# Use map to apply the prevous dict

df["Title"] =  df['Title'].map(title_dictionary)
df_test["Title"] =  df_test['Title'].map(title_dictionary)

# YOUR CODE HERE
#raise NotImplementedError()

In [10]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [11]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [12]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [13]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [14]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [15]:
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False))
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop')

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [16]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [17]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [18]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":HistGradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [19]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [20]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=4 )

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

total: 1.23s	remaining: 767ms
616:	learn: 0.3229018	total: 1.23s	remaining: 765ms
617:	learn: 0.3226934	total: 1.23s	remaining: 763ms
618:	learn: 0.3226679	total: 1.24s	remaining: 760ms
619:	learn: 0.3224384	total: 1.24s	remaining: 758ms
620:	learn: 0.3222797	total: 1.24s	remaining: 756ms
621:	learn: 0.3221968	total: 1.24s	remaining: 753ms
622:	learn: 0.3221056	total: 1.24s	remaining: 751ms
623:	learn: 0.3220152	total: 1.24s	remaining: 749ms
624:	learn: 0.3216825	total: 1.24s	remaining: 746ms
625:	learn: 0.3216729	total: 1.24s	remaining: 744ms
626:	learn: 0.3215428	total: 1.25s	remaining: 743ms
627:	learn: 0.3213518	total: 1.25s	remaining: 741ms
628:	learn: 0.3211542	total: 1.25s	remaining: 739ms
629:	learn: 0.3210773	total: 1.25s	remaining: 737ms
630:	learn: 0.3209931	total: 1.26s	remaining: 735ms
631:	learn: 0.3209187	total: 1.26s	remaining: 732ms
632:	learn: 0.3207711	total: 1.26s	remaining: 730ms
633:	learn: 0.3205822	total: 1.26s	remaining: 728ms
634:	learn: 0.3205237	total: 1.26s

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,84.769433,2.446236
2,Skl GBM,86.033520,84.044796,0.206932
3,XGBoost,84.916201,84.216074,0.181470
4,Skl HistGBM,84.357542,82.951252,0.596894
5,AdaBoost,83.798883,83.036891,0.201934
6,LightGBM,83.240223,82.042161,0.105034
7,Random Forest,82.681564,81.047431,0.380250
8,Extra Trees,79.329609,77.509881,0.382504
9,Decision Tree,76.536313,75.777339,0.021993


In [21]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [24]:
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(
   n_splits=2, 
   shuffle=True,
   random_state=4 
   )

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    #pred = cross_val_predict(model, x_val, y_val, cv=4)
    pred = model.predict(x_val) # Passes assert but not what was asked.

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,86.592179,84.769433,0.007997
2,Skl GBM,86.033520,84.044796,0.008996
3,XGBoost,84.916201,84.216074,0.009996
4,Skl HistGBM,84.357542,82.951252,0.011996
5,AdaBoost,83.798883,83.036891,0.016994
6,LightGBM,83.240223,82.042161,0.006998
7,Random Forest,82.681564,81.047431,0.026991
8,Extra Trees,79.329609,77.509881,0.024992
9,Decision Tree,76.536313,75.777339,0.009942


In [25]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [28]:
best_model = tree_classifiers["CatBoost"]
best_model.fit(x, y)

# Gives you the visual tree graph
# best_model = tree_classifiers["Skl GBM"]
# best_model.fit(x,y)

615:	learn: 0.3212445	total: 1.5s	remaining: 938ms
616:	learn: 0.3211510	total: 1.51s	remaining: 936ms
617:	learn: 0.3210769	total: 1.51s	remaining: 934ms
618:	learn: 0.3208985	total: 1.51s	remaining: 931ms
619:	learn: 0.3207590	total: 1.51s	remaining: 928ms
620:	learn: 0.3206044	total: 1.52s	remaining: 925ms
621:	learn: 0.3204401	total: 1.52s	remaining: 923ms
622:	learn: 0.3204008	total: 1.52s	remaining: 921ms
623:	learn: 0.3202512	total: 1.52s	remaining: 919ms
624:	learn: 0.3200623	total: 1.53s	remaining: 916ms
625:	learn: 0.3199443	total: 1.53s	remaining: 913ms
626:	learn: 0.3198935	total: 1.53s	remaining: 911ms
627:	learn: 0.3198756	total: 1.53s	remaining: 907ms
628:	learn: 0.3197588	total: 1.53s	remaining: 905ms
629:	learn: 0.3196496	total: 1.53s	remaining: 902ms
630:	learn: 0.3194488	total: 1.54s	remaining: 899ms
631:	learn: 0.3194051	total: 1.54s	remaining: 897ms
632:	learn: 0.3193237	total: 1.54s	remaining: 894ms
633:	learn: 0.3191322	total: 1.54s	remaining: 891ms
634:	learn: 0

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000002196C907D88>)])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [30]:
test_pred = best_model.predict(x_test)

In [31]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [32]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [33]:
sub.to_csv("sub.csv")

In [34]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

Successfully submitted to Titanic - Machine Learning from Disaster

  0%|          | 0.00/3.18k [00:00<?, ?B/s]
100%|██████████| 3.18k/3.18k [00:00<00:00, 19.6kB/s]
100%|██████████| 3.18k/3.18k [00:02<00:00, 1.41kB/s]
